In [18]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [ ]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [2]:
data = pd.read_excel('data/data_ford_price.xlsx') 

#  Отбор признаков: мотивация

## Предобработка данных

In [3]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [4]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [5]:
x.drop('lat', axis = 1, inplace = True)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [7]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

In [8]:
from sklearn.feature_selection import RFE

In [9]:
y = data['price']
x = data.drop(columns='price')

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [11]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [12]:
X_train.columns

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [13]:
selector.ranking_

array([1, 1, 4, 1, 3, 2])

##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [14]:
from sklearn.feature_selection import SelectKBest, f_regression

In [15]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

## Задание 9.5 

Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты. Загрузите полученный ноутбук (в формате IPYNB) в форму ниже.

In [16]:
# Сформируем датасеты с колонками которые были отобраны методом RFE и фильтрацией
RFE_columns = ['year', 'cylinders', 'lat']
f_reg_columns = ['year', 'cylinders', 'odometer']

X_RFE = data[RFE_columns]
y_RFE = data['price']

X_f_reg = data[f_reg_columns]
y_f_reg = data['price']

In [19]:
# разобьём данные на тренировочную и тестовый выборки для обоих методов

X_train_RFE, X_test_RFE, y_train_RFE, y_test_RFE = train_test_split(X_RFE, y_RFE, test_size=0.2, random_state= 42)

X_train_f_reg, X_test_f_reg, y_train_f_reg, y_test_f_reg = train_test_split(X_f_reg, y_f_reg, test_size=0.2, random_state= 42)

Модель линейной регрессии для RFE

In [20]:
RFE_model = LinearRegression()

RFE_model.fit(X_train_RFE, y_train_RFE)

y_pred_RFE = RFE_model.predict(X_test_RFE)

mae_RFE = mean_absolute_error(y_test_RFE, y_pred_RFE)

Модель линейной регрессии для SelectKBest

In [21]:
f_reg_model = LinearRegression()

RFE_model.fit(X_train_f_reg, y_train_f_reg)

y_pred_f_reg = RFE_model.predict(X_test_f_reg)

mae_f_reg = mean_absolute_error(y_test_f_reg, y_pred_f_reg)

In [23]:
print('MAE при использовании RFE {:.2f}'.format(mae_RFE))
print('MAE при использовании SelectKBest {:.2f}'.format(mae_f_reg))

MAE при использовании RFE 5280.02
MAE при использовании SelectKBest 4925.46


При сравнении MAE мы видим, что метод SelectKBest показал результат лучше чем RFE (меньше MAE -> Меньше ошибка -> Лучше модель)